In [4]:
!pip install langchain chromadb openai flask datasets evaluate torch transformers accelerate faiss-cpu


In [5]:
import pandas as pd
# Load the dataset into a pandas DataFrame
df = pd.read_csv(r'/content/counselchat-data.csv')

# Display first few rows
print(df.head())


                 questionID                                     questionTitle  \
0  5566fab2a64752d71ec3ca69  Escalating disagreements between mother and wife   
1  5566f94fa64752d71ec3ca64          I'm addicted to smoking. How can I stop?   
2  5567d26887a1cc0c3f3d8f46                    Keeping secrets from my family   
3  556bed15c969ba5861709df5         The Underlying Causes of Being Possessive   
4  556ba115c969ba5861709de6         Can I control anxiety without medication?   

                                        questionText  \
0  My wife and mother are having tense disagreeme...   
1  I'm planning to have baby, so I have to quit s...   
2  I have secrets in my mind, and I don't know wh...   
3  I am extremely possessive in my relationships ...   
4  I had a head injury a few years ago and my min...   

                                         questionUrl  \
0  https://counselchat.com/questions/escalating-d...   
1  https://counselchat.com/questions/i-m-addicted...   
2  https

In [6]:
# Display column names
print("Column Names:", df.columns.tolist())


Column Names: ['questionID', 'questionTitle', 'questionText', 'questionUrl', 'topics', 'therapistName', 'therapistUrl', 'answerText', 'upvotes']


In [7]:
# Extract relevant columns
questions = df["questionText"].dropna().tolist()  # Remove NaN values if any
answers = df["answerText"].dropna().tolist()

# Print a sample
print("Sample Question:", questions[0])
print("Sample Answer:", answers[0])


Sample Question: My wife and mother are having tense disagreements. In the past, they’ve had minor differences. For example, my wife would complain to me my mother is too overbearing; my mother would complain my wife is lazy.
However, it’s intensified lately. I think the cause is my wife talked back to her once. Now, any little disagreement is magnified, leading to major disagreements. What can I do?
Sample Answer: <p>What you are describing is something psychologists have termed &#34;triangulation&#34; which is what happens when one family member will not talk to the one they have a problem with and goes to a third member of the family to complain instead. You have been &#34;triangulated&#34; by your wife and mother.</p><p>This is often seen in families. It's seen everywhere. How many times have you had a problem with someone but you didn't go to them to tell them, you went to someone else to complain? It is usually difficult for a person to confront another, especially in relationshi

In [8]:
!pip install beautifulsoup4


In [9]:
import pandas as pd
from bs4 import BeautifulSoup

# Load the dataset into a pandas DataFrame
df = pd.read_csv(r'/content/counselchat-data.csv')

# Display column names and first few rows
print("Column Names:", df.columns.tolist())
print(df.head())

# Function to clean HTML tags from answers
def clean_html(text):
    return BeautifulSoup(text, "html.parser").get_text()

# Extract questions and clean answers
questions = df["questionText"].dropna().tolist()
answers = df["answerText"].dropna().apply(clean_html).tolist()

# Print a cleaned sample
print("Sample Question:", questions[0])
print("Sample Answer (Cleaned):", answers[0])


Column Names: ['questionID', 'questionTitle', 'questionText', 'questionUrl', 'topics', 'therapistName', 'therapistUrl', 'answerText', 'upvotes']
                 questionID                                     questionTitle  \
0  5566fab2a64752d71ec3ca69  Escalating disagreements between mother and wife   
1  5566f94fa64752d71ec3ca64          I'm addicted to smoking. How can I stop?   
2  5567d26887a1cc0c3f3d8f46                    Keeping secrets from my family   
3  556bed15c969ba5861709df5         The Underlying Causes of Being Possessive   
4  556ba115c969ba5861709de6         Can I control anxiety without medication?   

                                        questionText  \
0  My wife and mother are having tense disagreeme...   
1  I'm planning to have baby, so I have to quit s...   
2  I have secrets in my mind, and I don't know wh...   
3  I am extremely possessive in my relationships ...   
4  I had a head injury a few years ago and my min...   

                               

In [10]:
!pip install langchain openai


In [11]:
from langchain.prompts import PromptTemplate

# General Mental Health Support Prompt
general_prompt = PromptTemplate(
    template="""
You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: {client_message}
Therapist:
""",
    input_variables=["client_message"],
)

# Example usage:
client_message = "I've been feeling very anxious lately and don't know how to cope."
formatted_prompt = general_prompt.format(client_message=client_message)
print(formatted_prompt)



You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: I've been feeling very anxious lately and don't know how to cope.
Therapist:



In [12]:
def get_prompt(client_message):
    if any(word in client_message.lower() for word in ["anxious", "panic", "overwhelmed"]):
        return anxiety_prompt.format(client_message=client_message)
    elif any(word in client_message.lower() for word in ["depressed", "hopeless", "sad"]):
        return depression_prompt.format(client_message=client_message)
    elif any(word in client_message.lower() for word in ["relationship", "partner", "marriage", "family"]):
        return relationship_prompt.format(client_message=client_message)
    else:
        return general_prompt.format(client_message=client_message)

# Example Test
test_message = "I'm having serious conflicts with my wife, and I feel stuck."
selected_prompt = get_prompt(test_message)
print(selected_prompt)



You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: I'm having serious conflicts with my wife, and I feel stuck.
Therapist:



In [13]:
!pip install transformers torch accelerate


In [14]:
from huggingface_hub import login
login()


In [15]:
import requests

try:
    response = requests.get("https://huggingface.co", timeout=5)
    if response.status_code == 200:
        print("✅ Internet is working. Hugging Face is reachable!")
    else:
        print("⚠️ Internet issue: Received status code", response.status_code)
except requests.ConnectionError:
    print("❌ No internet connection detected!")


✅ Internet is working. Hugging Face is reachable!


In [2]:
!ping -c 3 huggingface.co


/bin/bash: line 1: ping: command not found


In [ ]:
import os
os.kill(os.getpid(), 9)  # Force restart the runtime


In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load Mistral-7B model & tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

print("Model loaded successfully!")


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Model loaded successfully!


In [18]:
def generate_response(client_message):
    # Format the structured prompt
    prompt = f"""
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: {client_message}
    Therapist:
    """

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200, temperature=0.7, top_p=0.9)

    # Decode and return the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# ✅ Test the chatbot
test_message = "I've been feeling very anxious lately and don't know how to cope."
response = generate_response(test_message)
print("\nChatbot Response:\n", response)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Chatbot Response:
 
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: I've been feeling very anxious lately and don't know how to cope.
    Therapist:
    
    I'm sorry to hear that you're feeling anxious. It's completely normal to experience anxiety at times, but it's important to address it when it becomes overwhelming.
    
    One helpful technique to manage anxiety is mindfulness meditation. This involves focusing on the present moment and accepting your thoughts and feelings without judgment. You can start by finding a quiet place to sit or lie down, closing your eyes, and focusing on your breath. As you breathe in and out, try to let go of any distracting thoughts and just be present in the moment.
    


In [19]:
!pip install datasets evaluate sacrebleu nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.4 MB/s eta 0:00:00


In [21]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=edc68959d3dcc217f563113ed721f4d3afa928bd7001a4cd18944a3cec891c16
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [22]:
import evaluate
import math
import torch
import nltk
from transformers import AutoModelForCausalLM, AutoTokenizer

nltk.download("punkt")  # Needed for tokenization in BLEU & ROUGE

# Load BLEU and ROUGE metrics from Hugging Face's Evaluate library
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# Function to calculate perplexity
def calculate_perplexity(response):
    inputs = tokenizer(response, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    perplexity = math.exp(loss.item())
    return perplexity


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
def evaluate_response(generated_response, reference_response):
    # Compute BLEU score (expects full sentences, not tokenized words)
    bleu_score = bleu.compute(predictions=[generated_response], references=[[reference_response]])

    # Compute ROUGE score (expects full sentences)
    rouge_score = rouge.compute(predictions=[generated_response], references=[reference_response])

    # Compute perplexity
    perplexity = calculate_perplexity(generated_response)

    return {
        "BLEU Score": bleu_score["bleu"],
        "ROUGE Score": rouge_score["rouge1"],
        "Perplexity": perplexity
    }


In [28]:
 import nltk
 nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [29]:
# Example chatbot response vs reference therapist response
generated_response = """
I'm sorry to hear that you're feeling anxious. It's completely normal to experience anxiety at times, but it's important to address it when it becomes overwhelming.
One helpful technique to manage anxiety is mindfulness meditation. This involves focusing on the present moment and accepting your thoughts and feelings without judgment.
"""
reference_response = """
I hear that you're feeling anxious, and that's completely understandable. A helpful way to manage anxiety is practicing deep breathing exercises or grounding techniques.
Taking a moment to focus on your breath or journaling can also be beneficial. Would you like to explore more coping strategies together?
"""

# Run evaluation
scores = evaluate_response(generated_response, reference_response)
print(scores)


{'BLEU Score': 0.12811300935224262, 'ROUGE Score': np.float64(0.38834951456310685), 'Perplexity': 2.6409495669289966}


In [30]:
!pip install faiss-cpu langchain chromadb


In [34]:
!pip install -U sentence-transformers



In [35]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load a free embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Free alternative to OpenAIEmbeddings

# Example dataset
conversation_texts = [
    "How do I deal with anxiety?",
    "What should I do if I'm feeling very anxious?",
    "How can I manage stress effectively?",
    "I've been feeling overwhelmed at work, how do I handle it?"
]

# Convert questions to vector embeddings
vectors = embedding_model.encode(conversation_texts, convert_to_numpy=True)

# Create FAISS index
vector_array = np.array(vectors, dtype=np.float32)
faiss_index = faiss.IndexFlatL2(vector_array.shape[1])
faiss_index.add(vector_array)

print("✅ FAISS Index Created Successfully with SentenceTransformers!")



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS Index Created Successfully with SentenceTransformers!


In [36]:
def retrieve_context(query):
    """Find similar past questions from FAISS and return relevant context."""
    query_vector = np.array([embedding_model.encode(query)], dtype=np.float32)
    distances, indices = faiss_index.search(query_vector, k=2)  # Retrieve top 2 similar questions

    # Fetch similar past questions
    similar_texts = [conversation_texts[i] for i in indices[0]]
    return similar_texts

# ✅ Test retrieval
query = "I'm feeling really stressed lately. What should I do?"
retrieved_context = retrieve_context(query)
print("🔍 Retrieved Context:", retrieved_context)


🔍 Retrieved Context: ['How can I manage stress effectively?', "What should I do if I'm feeling very anxious?"]


In [37]:
def generate_response_with_rag(client_message):
    # Retrieve similar past conversations
    context = retrieve_context(client_message)

    # Format prompt with retrieved knowledge
    prompt = f"""
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Context from previous conversations: {context}

    Client: {client_message}
    Therapist:
    """

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200, temperature=0.7, top_p=0.9)

    # Decode and return the response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ Test the chatbot with RAG
test_message = "I'm feeling really anxious lately. What should I do?"
response = generate_response_with_rag(test_message)
print("\nChatbot Response with RAG:\n", response)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Chatbot Response with RAG:
 
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Context from previous conversations: ["What should I do if I'm feeling very anxious?", 'How do I deal with anxiety?']

    Client: I'm feeling really anxious lately. What should I do?
    Therapist:
    
    I'm sorry to hear that you're feeling anxious. It's completely normal to experience anxiety at times, but it can be overwhelming and difficult to manage. Here are some suggestions that might help you cope with your anxiety:
    
    1. Practice mindfulness: Mindfulness is a technique that involves focusing on the present moment and accepting your thoughts and feelings without judgment. You can try mindfulness meditation, deep breathing exercises, or simply


In [ ]:
!pip install flask flask-cors
